In [ ]:
import cv2
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# import notebooks
import import_ipynb
from bg_remover_u2net import load_u2net, predict_mask, apply_mask
from bg_generator_sd import load_sd_model, generate_background

load models

In [ ]:
pipe = load_sd_model()
pipe.enable_attention_slicing()

model_path = '../u2net.pth'
net = load_u2net(model_path)

input image

In [ ]:
image_path = '../images/img1.jpg'

remove background

In [ ]:
mask, image = predict_mask(image_path, net)
foreground = apply_mask(image, mask)

In [ ]:
# original
plt.imshow(image)
plt.axis('off')
plt.show()

generate background

In [ ]:
# text prompt
prompt = "office with work desks"

In [ ]:
width, height = Image.fromarray(foreground).size

background = generate_background(prompt, width, height, pipe)

In [ ]:
background

blend foreground and background

In [ ]:
def blend(fg_rgba, bg_rgb, blur_amount=15):
    """
    fg_rgba: Foreground image with alpha channel (H, W, 4)
    bg_rgb: Background image without alpha (H, W, 3)
    """

    # Convert PIL images → NumPy for processing
    fg = np.array(fg_rgba).astype(float)
    bg = np.array(bg_rgb).astype(float)

    # Split channels
    alpha = fg[:, :, 3] / 255.0    # normalize 0–1
    alpha = alpha.astype(np.float32)

    # --- Soft Edge: Blur the alpha mask ---
    alpha = cv2.GaussianBlur(alpha, (blur_amount, blur_amount), 0)

    # Ensure sizes match
    alpha = alpha[:, :, None]  # make shape (H, W, 1)

    # Foreground RGB
    fg_rgb = fg[:, :, :3]

    # Blend: fg * alpha + bg * (1 - alpha)
    blended = fg_rgb * alpha + bg * (1 - alpha)
    blended = blended.astype(np.uint8)

    return Image.fromarray(blended)

In [ ]:
final_image = blend(foreground, background)

final output

In [ ]:
plt.imshow(final_image)
plt.axis('off')
plt.show()